# Assignment 2: Voting Visualized

## Deadline

Oct. 24th

## Important notes

- Make sure you push on GitHub your notebook with all the cells already evaluated.
- Note that maps do not render in a standard Github environment. You should export them to HTML and link them in your notebook.
- Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you implemented.
- Please write all your comments in English, and use meaningful variable names in your code.

## Background


* Are you curious to know what the political leanings of the people of Switzerland are?
* Do you wake up in a cold sweat, wondering which party won the last cantonal parliament election in Vaud?
* Are you looking to learn all sorts of visualizations, including maps, in Python?

If your answer to any of the above is yes, this assignment is just right for you. Otherwise, it's still an assignment, so we're terribly sorry.

The chief aim of this assignment is to familiarize you with visualizations in Python, particularly maps, and also to give you some insight into how visualizations are to be interpreted. The data we will use is the data on Swiss cantonal parliament elections from 2007 to 2018, which contains, for each cantonal election in this time period, the voting percentages for each party and canton.

For the visualization part, install [Folium](Folium) (_Hint: it is not available in your standard Anaconda environment, therefore search on the Web how to install it easily!_). Folium's README comes with very clear examples, and links to their own iPython Notebooks -- make good use of this information. For your own convenience, in this same directory you can already find one TopoJSON file, containing the geo-coordinates of the cantonal borders of Switzerland.

One last, general reminder: back up any hypotheses and claims with data, since this is an important aspect of the course.

In [1]:
import pandas as pd
import json
import folium
import os
import xlrd
import string


In [2]:
data_folder = './data/'

# Task 1: Cartography and census

__A)__ Display a Swiss map that has cantonal borders as well as the national borders. We provide a TopoJSON `data/ch-cantons.topojson.json` that contains the borders of the cantons.

__B)__ Take the spreadsheet `data/communes_pop.xls`, collected from [admin.ch](https://www.bfs.admin.ch/bfs/fr/home/statistiques/catalogues-banques-donnees/tableaux.assetdetail.5886191.html), containing population figures for every commune. You can use [pd.read_excel()](https://pandas.pydata.org/pandas-docs/version/0.20/generated/pandas.read_excel.html) to read the file and to select specific sheets. Plot a histogram of the population counts and explain your observations. Do not use a log-scale plot for now. What does this histogram tell you about urban and rural communes in Switzerland? Are there any clear outliers on either side, and if so, which communes?

__C)__ The figure below represents 4 types of histogram. At this stage, our distribution should look like Fig.(a). A common way to represent [power-laws](https://en.wikipedia.org/wiki/Power_law) is to use a histogram using a log-log scale  -- remember: the x-axis of an histogram is segmented in bins of equal sizes and y-values are the average of each bin. As shown in Fig.(b), small bins sizes might introduce artifacts. Fig.(b) and Fig.(c) are examples of histograms with two different bin sizes. Another great way to visualize such distribution is to use a cumulative representation, as show in Fig.(d), in which the y-axis represents the number of data points with values greater than y.  
  
Create the figures (b) and (d) using the data extracted for task 1B. For Fig.(b), represent two histograms using two different bin sizes and provide a brief description of the results. What does this tell you about the relationship between the two variables, namely the frequency of each bin and the value (i.e. population in case of the communal data) for each bin?

<img src="plaw_crop.png" style="width: 600px;">
  
The figure is extracted from [this paper](https://arxiv.org/pdf/cond-mat/0412004.pdf) that contains more information about this family of distributions.

## Task 2: Parties visualized

We provide a spreadsheet, `data/voters.xls`, (again) collected from [admin.ch](https://www.bfs.admin.ch/bfs/fr/home/statistiques/politique/elections/conseil-national/force-partis.assetdetail.217195.html), which contains the percentage of voters for each party and for each canton. For the following task, we will focus on the period 2014-2018 (the first page of the spreadsheet). Please report any assumptions you make regarding outliers, missing values, etc. Notice that data is missing for two cantons, namely Appenzell Ausserrhoden and Graubünden, and your visualisations should include data for every other canton.


__A)__ For the period 2014-2018 and for each canton, visualize, on the map, **the percentage of voters** in that canton who voted for the party [`UDC`](https://en.wikipedia.org/wiki/Swiss_People%27s_Party) (Union démocratique du centre). Does this party seem to be more popular in the German-speaking part, the French-speaking part, or the Italian-speaking part?

__B)__ For the same period, now visualize **the number of residents** in each canton who voted for UDC.

__C)__ Which one of the two visualizations above would be more informative in case of a national election with majority voting (i.e. when a party needs to have the largest number of citizens voting for it among all parties)? Which one is more informative for the cantonal parliament elections?

For part B, you can use the `data/national_council_elections.xslx` file ([guess where we got it from](https://www.bfs.admin.ch/bfs/fr/home/statistiques/politique/elections/conseil-national/participation.assetdetail.81625.html)) to have the voting-eligible population of each canton in 2015.

__Pre-Task Cleaning__

We see that our excel and csv data are quite messy, so before starting the exercices, we will clean it.


In [13]:
#We load the voters dataset
voters = pd.read_excel(data_folder + 'voters.xls')

#First we want to drop all the useless rows full of NaN, so we create a new dataframe for this purpose
voters_without_nan = voters.copy()

#The first column is temporarily removed, because often has the only nan value in a row. Without it, we can drop entire
#nan rows
voters_without_nan = voters_without_nan.drop(voters_without_nan.columns[0],axis=1)

#drop the NaN rows...
voters_without_nan=voters_without_nan.dropna(axis=0,how='all')

#We now have the indices of the rows without NaNs, so we only keep these in our original dataframe
voters=voters.iloc[voters_without_nan.index]

#Now we will drop the columns full of NaN
voters=voters.dropna(axis=1,how='all')

#The rest of NaN values corresponds to cantons where the political part is not present 
#i.e. there was 0 voters so we replace NaN by null
voters=voters.fillna(0)

#Method for cleaning a string (remove indexes, numbers, etc..)
def clean_name(name):
    if type(name) is str:
        #drop parethesis from party names
        name = name.replace(')', '')
        #strip digits from party names
        name = name.translate(name.maketrans('', '', string.digits))
    return name

#The first row of our dataframe now contains the party names. We clean them and make them our column names
#We rename the columns with corresponding labels
voters.iloc[0] = voters.iloc[0].map(clean_name)
voters = voters.rename(voters.iloc[0], axis = 1).rename(columns={0:'Cantons'})[1:]

#The first column of our table contains canton names. We clean them too.
voters.iloc[:,0] = voters.iloc[:,0].map(clean_name)

#Reset in the index in order to be able to concatenate our dataframe with cantons dataframe
voters.reset_index(drop=True,inplace = True)



#we load the cantons dataset
cantons = pd.read_csv(data_folder + 'cantons.csv')

#We join the two tables, because we see that the order of the cantons is the same in both 
voters_cantons = voters.join(cantons,sort = False)

#We convert the population data into floats in order to be able to do operations on it
#(To do this, we have to remove the commas and references at the end)

voters_cantons['Population']= voters_cantons['Population'].apply(lambda row: float(row.replace(",","")[:-4]))

#We create a new columns equal to the number of UDC voters (divide twice by 100 for percentages)
voters_cantons['UDC Voters'] = voters_cantons['Population']*voters_cantons['Participation']*voters_cantons['UDC']*0.0001
voters_cantons.drop('Canton of', axis = 1, inplace=True)
voters_cantons

,Cantons,Année électorale,Participation,PLR,PDC,PS,UDC,Dém.,PLS,AdI,...,Code,Since,Capital,Population,GDP percapita (2014)[16]in CHF,Area (km2),Density (per km2)[Note 4],No. munic.,Official languages,UDC Voters
0,Zurich,2015,32.6525,17.3278,4.87871,19.7164,30.0232,0,0,0,...,ZH,1351,Zürich,1504346.0,"96,411","1,729",701,168,German,147476
1,Berne,2018,30.5163,11.7179,0.671415,22.3288,26.7609,0,0,0,...,BE,1353,Bern,1031126.0,"76,307","5,960",158,352,"German, French",84206.2
2,Lucerne,2015,38.7413,21.0395,30.8625,11.8489,24.1156,0,0,0,...,LU,1332,Lucerne,403397.0,"65,119","1,494",233,83,German,37688.2
3,Uri,2016,61.9891,26.8567,31.2988,12.985,24.0532,0,0,0,...,UR,1291,Altdorf,36145.0,"51,332","1,077",33,20,German,5389.34
4,Schwytz,2016,37.7471,21.629,27.1677,12.9254,33.1151,0,0,0,...,SZ,1291,Schwyz,155863.0,"58,788",908,143,30,German,19482.8
5,Obwald,2018,53.7933,17.1747,29.8036,15.0909,24.5323,0,0,0,...,OW,1291,Sarnen,37378.0,"64,253",491,66,7,German,4932.68
6,Nidwald,2018,54.9216,28.0124,26.7501,4.47005,25.9166,0,0,0,...,NW,1291,Stans,42556.0,"69,559",276,138,11,German,6057.35
7,Glaris,2018,29.4897,18.3928,9.40085,12.7504,25.2754,0,0,0,...,GL,1352,Glarus,40349.0,"67,379",685,51,3,German,3007.46
8,Zoug,2014,42.9394,22.1479,26.7831,9.25091,23.6318,0,0,0,...,ZG,1352,Zug,123948.0,"150,613",239,416,11,German,12577.5
9,Fribourg,2016,39.3021,18.1655,23.707,23.5843,19.7176,0,0,0,...,FR,1481,Fribourg,315074.0,"58,369","1,671",141,150,"French, German",24416.5


In [14]:
#Data visualization for UDC vote percentage by canton
cantons_data= json.load(open(data_folder + 'ch-cantons.topojson.json'))

m = folium.Map([47, 8.33],tiles='cartodbpositron', zoom_start=7)

m.choropleth(
 geo_data=cantons_data,
 topojson='objects.cantons',   
 name='choropleth',
 data=result_data,
 columns=['Code', 'UDC'],
 key_on='feature.id',
 fill_color='YlGn',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='UDC vote Rate (%)'
)
folium.LayerControl().add_to(m)

m.save('UDC vote rate.html')
m

NameError: name 'result_data' is not defined

In [ ]:
#Data visualization for number of UDC voters by canton

m = folium.Map([47, 8.33],tiles='cartodbpositron', zoom_start=7)


m.choropleth(
 geo_data=cantons_data,
 topojson='objects.cantons',   
 name='choropleth',
 data=result,
 columns=['Code', 'UDC voters'],
 key_on='feature.id',
 fill_color='YlGn',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='Number of UDC voters',
 threshold_scale=[0,10000,30000,60000, 90000,120000]
)
folium.LayerControl().add_to(m)
m.save('UDC voters number.html')
m

In [ ]:
#Does this party seem to be more popular in the German-speaking part, the French-speaking part, or the Italian-speaking part?
#ATTENTION J'AI EU BCP DE MAL A FAIRE CETTE MAP QUI N'EST PAS DEMANDEE ! DU COUP C'EST PAS OPTI


german_part = result['Official languages'] == 'German'
french_part = result['Official languages'] == 'French'
italian_part = result['Official languages'] == 'Italian'

result['Population']=result['Participation']*result['Population']/100

german_swiss_UDC_popularity=result[german_part]['UDC voters'].sum()/result[german_part]['Population'].sum()
french_swiss_UDC_popularity=result[french_part]['UDC voters'].sum()/result[french_part]['Population'].sum()
italian_swiss_UDC_popularity=result[italian_part]['UDC voters'].sum()/result[french_part]['Population'].sum()

result_french=result[french_part]

result_german=result[german_part]

result_italian=result[italian_part]
result

In [ ]:
result_french.loc[:,'Participation']=french_swiss_UDC_popularity
result_german.loc[:,'Participation']=german_swiss_UDC_popularity
result_italian.loc[:,'Participation']=italian_swiss_UDC_popularity

print(french_swiss_UDC_popularity)
print(german_swiss_UDC_popularity)
print(italian_swiss_UDC_popularity)

In [ ]:
frames = [result_french,result_german,result_italian]
frenchvsgermanpopularity=pd.concat(frames)
frenchvsgermanpopularity

In [ ]:
result['Participation'].iloc[frenchvsgermanpopularity.index]=frenchvsgermanpopularity['Participation']*100

result.loc [[1,9,22],['Participation']]= 0.0
result

In [ ]:
m = folium.Map([47, 8.33],tiles='cartodbpositron', zoom_start=7)



m.choropleth(
 geo_data=cantons_data,
 topojson='objects.cantons',   
 name='choropleth',
 data=result,
 columns=['Code', 'Participation'],
 key_on='feature.id',
 fill_color='YlGn',
 fill_opacity=0.7,
 line_opacity=0.1,
 legend_name='UDC vote Rate in german and french speaking Switzerland(%)',
 threshold_scale=[0,5,10,15,20,25]
 
)
folium.LayerControl().add_to(m)
m.save('UDC vote rate GERvsFR.html')
m

Which one of the two visualizations above would be more informative in case of a national election with majority voting (i.e. when a party needs to have the largest number of citizens voting for it among all parties)? Which one is more informative for the cantonal parlement elections?

In case of a national election with majority voting it is clearly the visualization of part B with the raw voters number that is the more usefull. The only problem is that the reader need to have an idea of the voters total population in order to understand the numbers on this map. It is good to be able to see that UDC can have 30 000 voters in one canton but if the PS have 50 000 UDC will not win despite the impressive number... Thus, percentage is often more clear unlike my explanation.

On the other hand, for the cantonal parlement elections, it is the popularity of the party by cantons that will be important and the visualization of part A.